# Importando Série Histórica do BTC

In [30]:
import json
import requests
import pandas as pd

# insert your API key here
API_KEY = '2DW9AJS0I2CjieLcF9nhVeDVU79'

# make API request
res = requests.get('https://api.glassnode.com/v1/metrics/market/price_usd_ohlc', params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
btc = pd.read_json(res.text, convert_dates=['t'])
btc.rename(columns = {"t" : "Data", "o" : "BTC"}, inplace = True)
pd.to_datetime(btc["Data"], format='%d %b %Y')
btc = btc.from_records(btc["BTC"].tolist()).join(btc).drop("BTC", axis = 1).set_index("Data")
btc.rename(columns = {"o" : "Abertura", "h" : "Máxima", "l" : "Mínima", "c" : "Fechamento"}, inplace = True)
btc.to_excel("serie_bitcoin.xlsx")
btc.head()

,Fechamento,Máxima,Mínima,Abertura
Data,,,,
2010-07-17,0.049510,0.04951,0.04951,0.04951
2010-07-18,0.085840,0.08585,0.05941,0.05941
2010-07-19,0.080800,0.09307,0.07855,0.09090
2010-07-20,0.074733,0.08181,0.07426,0.08181
2010-07-21,0.079210,0.07921,0.06634,0.07425


# Importando Indicadores

In [36]:
df = pd.DataFrame()

In [37]:
indicadores_luis = pd.read_excel("Indicadores Luis.xlsx")
indicadores_luis.index = indicadores_luis["Data"]
indicadores_luis.drop("Data", axis = 1, inplace = True)
indicadores_luis.head()

,MayerMutiple,BPT,DeltaGradient140,DeltaGradient28,MVRV,MVRVZ,Thermocap
Data,,,,,,,
2010-07-18,NaN,NaN,NaN,NaN,146.038332,2.518053,5.025170e-07
2010-07-19,NaN,NaN,NaN,NaN,62.565130,2.097027,4.708351e-07
2010-07-20,NaN,NaN,NaN,NaN,45.106630,1.651545,4.336639e-07
2010-07-21,NaN,NaN,NaN,NaN,38.357560,1.927019,4.577077e-07
2010-07-22,NaN,NaN,NaN,NaN,25.132348,0.885502,3.221589e-07


In [38]:
indicadores_samu = pd.read_excel("Indicadores Samuel.xlsx")
indicadores_samu.index = indicadores_samu["Data"]
indicadores_samu.drop("Data", axis = 1, inplace = True)

df = indicadores_samu.join(indicadores_luis)
indicadores_samu.head()

,Daily Inflation Ratio,Annual Inflation Rate,Active Addresses,Hash Rate 30 Day,Median Transaction Size,MayerMutiple,BPT,DeltaGradient140,DeltaGradient28,MVRV,MVRVZ,Thermocap
Data,,,,,,,,,,,,
2009-01-03,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-04,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-05,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-06,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-07,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Importando do Google Trends

In [6]:
def trends_import(palavra):
    import pytrends 
    from pytrends.request import TrendReq
    import pandas as pd
    import datetime
    
    pytrends = TrendReq(hl='en-US', tz=360)
    # build payload
    kw_list = [palavra] # list of keywords to get data 
    pytrends.build_payload(kw_list, cat=0, timeframe='today 12-m')
    data = pytrends.get_historical_interest(kw_list,
                                            year_start=2009, month_start=1, day_start=1, hour_start=0,
                                            year_end=2021, month_end=12, day_end=30, hour_end=0,
                                            cat=0, sleep=0, frequency = "daily")
    return data

In [7]:
trends = trends_import("Bitcoin")
trends.drop("isPartial", axis = 1, inplace = True)
trends.columns = ["Pesquisas"]
df = df.join(trends)
trends.head()

,Pesquisas
date,
2009-01-01,76
2009-01-02,0
2009-01-03,0
2009-01-04,0
2009-01-05,0


# Importando do Glassnode

In [8]:
def glassnode_import(indicador):

    import json
    import requests
    import pandas as pd

    # insert your API key here
    API_KEY = '2DW9AJS0I2CjieLcF9nhVeDVU79'

    # make API request
    res = requests.get(f'https://api.glassnode.com/v1/metrics/indicators/{indicador}', params={'a': 'BTC', 'api_key': API_KEY})

    # convert to pandas dataframe
    df = pd.read_json(res.text, convert_dates=['t'])
    df.rename(columns = {"t" : "date", "v" : indicador}, inplace = True)
    df.index = df["date"]
    df.drop("date", axis = 1, inplace = True)
    return df

In [42]:
sopr = glassnode_import("sopr")
unrealized_profit = glassnode_import("unrealized_profit")
cdd = glassnode_import("cdd")

In [43]:
stock_to_flow_ratio = glassnode_import("stock_to_flow_ratio")
stock_to_flow_ratio.reset_index(inplace = True)
stock_to_flow_ratio = stock_to_flow_ratio.from_records(stock_to_flow_ratio["o"].tolist()).join(stock_to_flow_ratio).drop("o", axis = 1)
stock_to_flow_ratio.set_index("date", inplace = True)

stock_to_flow_ratio["DELTA_STF"] = stock_to_flow_ratio["ratio"] - stock_to_flow_ratio["price"]
stock_to_flow_ratio.drop(["ratio", "price"], axis = 1, inplace = True)

In [44]:
# Adicionando todos os indicadores na base
nomes = [sopr, unrealized_profit, cdd, stock_to_flow_ratio]
for i in nomes: df = df.join(i)

# Exportando Indicadores

In [45]:
# Nomes antigos das variáveis
dados = df.copy()
dados.columns.value_counts()

unrealized_profit          1
Annual Inflation Rate      1
cdd                        1
Median Transaction Size    1
Hash Rate 30 Day           1
Daily Inflation Ratio      1
sopr                       1
Pesquisas                  1
MVRVZ                      1
DELTA_STF                  1
daysTillHalving            1
Active Addresses           1
DeltaGradient28            1
Thermocap                  1
MayerMutiple               1
DeltaGradient140           1
MVRV                       1
BPT                        1
dtype: int64

In [46]:
# Renomeando indicadores conforme documentação (https://docs.google.com/spreadsheets/d/1g3A679ed7p6Q23rRea6Aav3RPhSiXnrP/edit#gid=1457644454)
dados.rename(columns = {
                     "Hash Rate 30 Day" : "HASH_RATE30",
                     "Active Addresses" : "END_ATIVOS",
                     "Daily Inflation Ratio" : "DY_INFLATION",
                     "Annual Inflation Rate" : "AN_INFLATION",
                     "Median Transaction Size" : "FEE_MEAN",
                     #HODL_W
                     "daysTillHalving" : "DAYS_HALVING",
                     "DELTA_STF" : "DELTA_STF",
    
                     "Pesquisas" : "TRENDS",
                     "sopr" : "SOPR",
                     #LTH
                     "unrealized_profit" : "UNR_PROFIT",
                     "cdd" : "CDD",
    
                     "MayerMutiple" : "MMULT",
                     "MVRV" : "MVRV",
                     "MVRVZ" : "MVRV_Z",
                     "Thermocap" : "THERMOCAP",
                     "DeltaGradient28" : "MRGO28",
                     "DeltaGradient140" : "MRGO140",
                     "BPT" : "BPT"
                     }, inplace = True)

# Apagando indicadores excluídos
dados.drop(["BPT"], axis = 1, inplace = True)

In [48]:
dados.columns

Index(['DY_INFLATION', 'AN_INFLATION', 'END_ATIVOS', 'HASH_RATE30', 'FEE_MEAN',
       'MMULT', 'MRGO140', 'MRGO28', 'MVRV', 'MVRV_Z', 'THERMOCAP', 'TRENDS',
       'SOPR', 'UNR_PROFIT', 'CDD', 'DAYS_HALVING', 'DELTA_STF'],
      dtype='object')

In [47]:
dados.describe()

,DY_INFLATION,AN_INFLATION,END_ATIVOS,HASH_RATE30,FEE_MEAN,MMULT,MRGO140,MRGO28,MVRV,MVRV_Z,THERMOCAP,TRENDS,SOPR,UNR_PROFIT,CDD,DAYS_HALVING,DELTA_STF
count,4987.000000,4987.000000,4.993000e+03,4.958000e+03,4.430000e+03,4002.000000,3924.000000,4036.000000,4201.000000,4064.000000,4.064000e+03,4763.000000,4431.000000,4074.000000,4.628000e+03,4628.000000,4431.000000
mean,0.194861,71.124225,4.199272e+05,3.661235e+07,5.154456e+01,1.423861,0.917901,0.572580,2.082196,1.700410,8.085980e-07,27.083981,1.009172,0.493426,6.944673e+06,710.379862,7566.623096
std,2.168322,791.437568,3.777737e+05,5.991494e+07,6.083544e+01,0.992737,2.723995,2.792712,2.874091,1.895728,7.157282e-07,18.196559,0.045206,0.128989,1.280898e+07,387.568725,20949.896502
min,0.001934,0.705911,0.000000e+00,2.119314e-06,8.203471e-08,0.225876,-7.566771,-10.531650,0.386829,-0.830057,1.202472e-07,0.000000,0.647491,0.018593,0.000000e+00,1.000000,-14014.049323
25%,0.010290,3.755810,2.879900e+04,1.294414e+01,4.604806e+00,0.904389,-0.534868,-0.617850,1.286646,0.469422,3.280976e-07,13.000000,0.997115,0.403039,1.799755e+06,385.000000,-98.099009
50%,0.024925,9.097482,3.576350e+05,4.605822e+05,2.156920e+01,1.169336,0.390942,0.143740,1.743184,1.230927,5.737692e-07,27.000000,1.002395,0.474880,3.967926e+06,732.000000,-0.073230
75%,0.079760,29.112404,7.411840e+05,4.996068e+07,8.751409e+01,1.582944,1.856018,1.176056,2.410370,2.301056,9.953135e-07,37.000000,1.011054,0.581284,7.987499e+06,1020.000000,1423.760566
max,100.000000,36500.000000,1.366494e+06,2.216993e+08,5.348536e+02,13.296906,16.228291,22.476845,146.038332,12.547296,6.385165e-06,100.000000,1.896209,0.877967,3.971068e+08,1448.000000,88101.052881


In [50]:
dados.head()

,DY_INFLATION,AN_INFLATION,END_ATIVOS,HASH_RATE30,FEE_MEAN,MMULT,MRGO140,MRGO28,MVRV,MVRV_Z,THERMOCAP,TRENDS,SOPR,UNR_PROFIT,CDD,DAYS_HALVING,DELTA_STF
2009-01-03,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2009-01-04,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2009-01-05,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2009-01-06,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2009-01-07,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [53]:
dados.to_excel("indicadores.xlsx", index_label = "Data")